In [1]:
scale_cadence = 3

from pygeodyn.PYGEODYN import Pygeodyn
import datetime


scaling_cadence = 3
scale_cadence = scaling_cadence
index_buffarc_start = 0
run_list = [
                'msis2',
#                 'jb2008',
                'dtm2020_o',
#                 'hasdm_oc'
           ]

year_list = [2018]
# month_list = ['jan', 'feb', 'mar', 'apr','may', 'jun','jul','aug','sep', 'oct', 'nov','dec' ]

month_list = ['oct','nov']#, 'nov','dec']



dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    for y_num in year_list: #, 
        for imonth,month in enumerate(month_list):
            if y_num==2018 and month not in ['oct', 'nov', 'dec']:
                print(f'Skipping {y_num}.{month}')
                continue
            if y_num==2019 and month not in ['jan', 'feb', 'mar', 'apr']:
                print(f'Skipping {y_num}.{month}')
                continue

            if i =='msis2':
                run_dict[str(y_num)+month+i]={}
                run_dict[str(y_num)+month+i]['num'] = 5
                run_dict[str(y_num)+month+i]['model_path'] = None
            if i =='dtm2020_o':
                run_dict[str(y_num)+month+i]={}
                run_dict[str(y_num)+month+i]['num'] = 3
                run_dict[str(y_num)+month+i]['model_path'] = None
            if i =='jb2008':
                run_dict[str(y_num)+month+i]={}
                run_dict[str(y_num)+month+i]['num'] = 1
                run_dict[str(y_num)+month+i]['model_path'] = None
            if i =='hasdm_oc':
                run_dict[str(y_num)+month+i]={}
                run_dict[str(y_num)+month+i]['num'] = 2 
                run_dict[str(y_num)+month+i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' #HASDM_OrbitCloud_2018313.01.csv


print(run_dict)

{'2018octmsis2': {'num': 5, 'model_path': None}, '2018novmsis2': {'num': 5, 'model_path': None}, '2018octdtm2020_o': {'num': 3, 'model_path': None}, '2018novdtm2020_o': {'num': 3, 'model_path': None}}


## Load Pickles

In [2]:

import pandas as pd
from datetime import datetime,timedelta
import numpy as np

import pandas as pd
from datetime import datetime,timedelta
import numpy as np



arc_timesfile = '/data/SatDragModelValidation/data/inputs/raw_inputdata/data_ICESat2/arc_times.txt'

### Make a function that finds the right range of dates in this file.
arcs = pd.read_csv(arc_timesfile, 
            sep = ',',
#             dtype=object,
            names = [
                'arc'         ,
                'epoch_start' ,
                'epoch_stop'  ,
                'orbit_start' ,
                'orbit_stop'  ,
                    ],)


arcs_yyyyddd = [x.strip() for x in arcs['arc'].values.tolist()]
epochstart   = [x.strip() for x in arcs['orbit_start'].values.tolist()]
epochstop    = [x.strip() for x in arcs['orbit_stop' ].values.tolist()]


input_arcs       = []
input_epochstart = []
input_epochstop  = []
arc_length = []
scaleparameter_times = []

for y_num in year_list:

    for imonth,month in enumerate(month_list):
        if month=='jan':  m_num = 1
        if month=='feb':  m_num = 2
        if month=='mar':  m_num = 3
        if month=='apr':  m_num = 4
        if month=='may':  m_num = 5
        if month=='jun':  m_num = 6
        if month=='jul':  m_num = 7
        if month=='aug':  m_num = 8
        if month=='sep':  m_num = 9
        if month=='oct':  m_num = 10
        if month=='nov':  m_num = 11
        if month=='dec':  m_num = 12




        for i,val in enumerate(arcs_yyyyddd):

            if pd.to_datetime(val[:8],format="%Y.%j").month == m_num and pd.to_datetime(val[:8],format="%Y.%j").year == y_num:
                epoch_delta  =  pd.to_timedelta(pd.to_datetime(epochstop[i]) - pd.to_datetime(epochstart[i]), 'hours')
                arc_length.append(epoch_delta.total_seconds()/3600)

                if val[8:] == 'A' or val[8:] == 'B':
                    input_arcs.append(      arcs_yyyyddd[i])
                    input_epochstart.append(epochstart[i])
                    input_epochstop.append( epochstop[i])
    #                 scaleparameter_times.append(dict_sftime[arcs_yyyyddd[i]])
                    scalestart = str(pd.to_datetime(arcs_yyyyddd[i][:-1],format="%Y.%j"))
                    scalestop  = str(pd.to_datetime(arcs_yyyyddd[i][:-1],format="%Y.%j") + pd.to_timedelta(24,'h'))
                    scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                              end   = pd.to_datetime( epochstop[i] ),
                                                              freq=str(scale_cadence)+"h")
                    
                    if epochstop[i] in scaletimes:  ## dont add repeats to the final epoch time
                        ### Append the epoch end time to the end
                        add_epochend = scaletimes.values.astype(np.int64) // 10 ** 9
                                        #np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                      #pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)

                        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                            format ='%y%m%d%H%M%S.%f' )
                                         for ts in add_epochend  ])


                    else:
                        ### Append the epoch end time to the end
                        add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                      pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)


                        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                            format ='%y%m%d%H%M%S.%f' )
                                         for ts in add_epochend  ])

                    pass
                else:

                    ### Use Arc text file times
                    input_arcs.append(      arcs_yyyyddd[i])
                    input_epochstart.append(epochstart[i])
                    input_epochstop.append( epochstop[i])
    #                 print(f"epoch_top {epochstop[i]}")
                    scalestart = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j"))
                    scalestop  = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j") + pd.to_timedelta(24,'h'))
                    scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                              end   = pd.to_datetime(scalestop ),
                                                              freq=str(scale_cadence)+"h")

                    ### Append the epoch end time to the end
                    add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                  pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)


                    scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                        format ='%y%m%d%H%M%S.%f' )
                                     for ts in add_epochend ])
# for i,val in enumerate(input_arcs):
#     print(f" {input_arcs[i]:10}  , {input_epochstart[i]:20},  {input_epochstop[i]:20}, {arc_length[i]}")
# #     if val[8:] == 'A' or val[8:] == 'B':
#     print(f" {'':10}  , {scaleparameter_times[i]}")
#     print("") 



In [3]:
def setsettings(den, y_num, month, run_dict, scaling_cadence, file_raw_ICs, file_g2b, input_arcs, input_epochstart, input_epochstop, scaleparameter_times):
    
    settings_ice= {# Basic input settings
                 'satellite'      : {'input': 'icesat2'},
                 'den_model'      : {'input': den},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'run_specifier'  : {'input': '_'+str(y_num)+month},
                 'cd_model'       : {'input': 'DRIA'},
                 'file_string'    : {'input': 'DRIAscaled_'},
                 'model_data_path' : {'input': run_dict[str(y_num)+month+den]['model_path']},
                 # Force Model settings
                  'cd_type'               : {'input':"Fixed_CD"},
                  'cd_value'              : {'input':1.0 },
                  'scaling_factor'        : {'input':True},
                  'hours_between_cd_adj'  : {'input':scaling_cadence},
                  #### Comment for unadjusted run:
                  'cd_adjustment_boolean' : {'input':True },
                #### DRIA CD Model Parameters
                'cd_model_params' : {'input':{ 
                        'MS'     : '26.980D0'   ,  #!  molar mass for each panel (g/mol)
                        'TW'     : '300.0D0'    ,  #!  temperature of panels  (K)
                           ###  Alpha is b/w 0 and 1
                        'ALPHA'  : '0.890D0'    ,  #!  accomodation coefficient
                        'KL'     : '0.0D0'    ,    #!  langmuir parameter
                        'FRACOX' : '1.0D0'   ,     #!  fraction of surface covered by atomic oxygen
                   }},
                  #### ---------------------------------------
                 # Run
                  'arc_type'       : {'input':'Nominal30hr_and_AB'},      
                  'step'           : {'input': 10},
                  'orbfil_step'    : {'input': 120.},
                  'which_ICfile'   : {'input':file_raw_ICs},
                  'which_g2bfile'  : {'input':file_g2b},
                    #
                  'arc'            : {'input': input_arcs},
                  'epoch_start'    : {'input': input_epochstart},
                  'epoch_stop'     : {'input': input_epochstop},  
            'scaleparameter_times' : {'input': scaleparameter_times},  
                   #                                
                  'global_options' : {'input':'pso_2018'},
                 # Request read on raw outputs
                  'request_data'   : {'input': ['Trajectory_orbfil', 
                                               'Density', 
                                               'Residuals_summary',
                                               'DragFile',
                                               'AdjustedParams'
                                               ]},
              #end dict
              }
    return(settings_ice)


In [4]:
%load_ext autoreload
%autoreload 2
from gc import collect as gc_collect
import pickle 
import os
from datetime import datetime,timedelta


g2b_path = "/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"

dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'icesat2/FinalDatasets_SWJ2025/'

obj = {}

for y_num in year_list: #

    for imonth,month in enumerate(month_list):
        if month=='jan':  m_num = 1
        if month=='feb':  m_num = 2
        if month=='mar':  m_num = 3
        if month=='apr':  m_num = 4
        if month=='may':  m_num = 5
        if month=='jun':  m_num = 6
        if month=='jul':  m_num = 7
        if month=='aug':  m_num = 8
        if month=='sep':  m_num = 9
        if month=='oct':  m_num = 10
        if month=='nov':  m_num = 11
        if month=='dec':  m_num = 12
            
        if y_num==2018 and m_num<10:
            print(f'Skipping {y_num}.{m_num}')
            continue
        if y_num==2019 and m_num>4:
            print(f'Skipping {y_num}.{m_num}')
            continue
            
            
        file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_{y_num}_{m_num:02d}.txt"
        file_g2b     = f"pce_icesat2_pso_{y_num}_{m_num:02d}"
    #     file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_20190331_20190503.txt"
    #     file_g2b     = f"pce_icesat2_pso_20190331_20190503"

        input_arcs           = []
        input_epochstart     = []
        input_epochstop      = []
        arc_length           = []
        scaleparameter_times = []

        for i,val in enumerate(arcs_yyyyddd):


            if pd.to_datetime(val[:8],format="%Y.%j").month == m_num and pd.to_datetime(val[:8],format="%Y.%j").year == y_num:
                epoch_delta  =  pd.to_timedelta(pd.to_datetime(epochstop[i]) - pd.to_datetime(epochstart[i]), 'hours')
                arc_length.append(epoch_delta.total_seconds()/3600)

                if val[8:] == 'A' or val[8:] == 'B':
                    input_arcs.append(      arcs_yyyyddd[i])
                    input_epochstart.append(epochstart[i])
                    input_epochstop.append( epochstop[i])
    #                 scaleparameter_times.append(dict_sftime[arcs_yyyyddd[i]])
                    scalestart = str(pd.to_datetime(arcs_yyyyddd[i][:-1],format="%Y.%j"))
                    scalestop  = str(pd.to_datetime(arcs_yyyyddd[i][:-1],format="%Y.%j") + pd.to_timedelta(24,'h'))
                    scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                              end   = pd.to_datetime( epochstop[i] ),
                                                              freq=str(scale_cadence)+"h")
                
                    if epochstop[i] in scaletimes:  ## dont add repeats to the final epoch time
                        ### Append the epoch end time to the end
                        add_epochend = scaletimes.values.astype(np.int64) // 10 ** 9
                                        #np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                      #pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)

                        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                            format ='%y%m%d%H%M%S.%f' )
                                         for ts in add_epochend  ])
                    else:
                        ### Append the epoch end time to the end
                        add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                      pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)


                        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                            format ='%y%m%d%H%M%S.%f' )
                                         for ts in add_epochend ])
                else:

                    ### Use Arc text file times
                    input_arcs.append(      arcs_yyyyddd[i])
                    input_epochstart.append(epochstart[i])
                    input_epochstop.append( epochstop[i])
    #                 print(f"epoch_top {epochstop[i]}")
                    scalestart = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j"))
                    scalestop  = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j") + pd.to_timedelta(24,'h'))
                    scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                              end   = pd.to_datetime(scalestop ),
                                                              freq=str(scale_cadence)+"h")

                    ### Append the epoch end time to the end
                    add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                  pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)


                    scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                        format ='%y%m%d%H%M%S.%f' )
                                     for ts in add_epochend ])

        for i,val in enumerate(input_arcs):
            print(f" {input_arcs[i]:10}  , {input_epochstart[i]:20},  {input_epochstop[i]:20}, {arc_length[i]}")
#             if val[8:] == 'A' or val[8:] == 'B':
#                 print(f" {'':10}  , {scaleparameter_times[i]}")
#                 print("")
#             else:
#                 print("")     
    

    for i,model in enumerate(run_list):
        for imonth,month in enumerate(month_list):
            
            print('y_num month',y_num, month)
            if y_num==2018 and month not in ['oct', 'nov', 'dec']:
                print(f'Skipping {y_num}.{month}')
                
            elif y_num==2019 and month not in ['jan', 'feb', 'mar', 'apr']:
                print(f'Skipping {y_num}.{month}')
                
            else:
                pickleName = f'_{y_num}.{month}_DRIA_scale{scale_cadence}.pkl'

                ### Load the data if the pickles exist
                print()
                print()
                gc_collect()

                pickle_file = dir_save+model+pickleName

                obj[str(y_num)+month+model] = pd.read_pickle(pickle_file)
        #         filehandler = open(pickle_file, 'rb') 
        #         obj[month+model] = pickle.load(filehandler)
        #         filehandler.close()
                print('Loaded data from pickle... ',y_num, month, model)

### Save space if doing density retrieval
for model in run_dict.keys():
    print(model)
    del obj[model]['OrbitResids']
    del obj[model]['Trajectory_orbfil']

gc_collect()

 2018.288    , 2018-10-14 21:18:00 ,  2018-10-16 02:42:00 , 29.4
 2018.289A   , 2018-10-15 21:18:00 ,  2018-10-16 23:00:00 , 25.7
 2018.290A   , 2018-10-17 00:24:00 ,  2018-10-18 02:42:00 , 26.3
 2018.291    , 2018-10-17 21:18:00 ,  2018-10-19 02:42:00 , 29.4
 2018.292    , 2018-10-18 21:18:00 ,  2018-10-20 02:42:00 , 29.4
 2018.293    , 2018-10-19 21:18:00 ,  2018-10-21 02:42:00 , 29.4
 2018.294    , 2018-10-20 21:18:00 ,  2018-10-22 02:42:00 , 29.4
 2018.295    , 2018-10-21 21:18:00 ,  2018-10-23 02:42:00 , 29.4
 2018.296    , 2018-10-22 21:18:00 ,  2018-10-24 02:42:00 , 29.4
 2018.297    , 2018-10-23 21:18:00 ,  2018-10-25 02:42:00 , 29.4
 2018.298    , 2018-10-24 21:18:00 ,  2018-10-26 02:42:00 , 29.4
 2018.299    , 2018-10-25 21:18:00 ,  2018-10-27 02:42:00 , 29.4
 2018.300    , 2018-10-26 21:18:00 ,  2018-10-28 02:42:00 , 29.4
 2018.301A   , 2018-10-27 21:18:00 ,  2018-10-28 16:00:00 , 18.7
 2018.301B   , 2018-10-28 17:36:00 ,  2018-10-29 02:42:00 , 9.1
 2018.302A   , 2018-10-28 

0

In [5]:

def get_continuous_scaled_densities(OBJECT, model_dict, choose_model, scale_cadence):
    den_dict                =   {}
    den_dict['dates'    ]   =   []
    den_dict['lon'      ]   =   []
    den_dict['lat'      ]   =   []
    den_dict['alt'      ]   =   []
#     den_dict['denscaled']   =   []
    den_dict['dens']   =   []
    den_dict['CD']   =   []

    # if choose_model == 'novmsis2':
    #     ARCSET = OBJECT[choose_model]['global_params']['arc_input']
    #     print(ARCSET)
    for ii,arc in enumerate(OBJECT[choose_model]['global_params']['arc_input']):
        epochstart = OBJECT[choose_model]['global_params']['prms']['epoch_start'][ii]
        epochstop = OBJECT[choose_model]['global_params']['prms']['epoch_stop'][ii]
        hrs        = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
        frachours  =(hrs/24)
        #
        if len(arc) == 9:
            maneuv_indicator = arc[8]
        else:
            maneuv_indicator = ''

        arc_type = OBJECT[choose_model]['global_params']['prms']['arc_type']
        if arc_type == "Nominal30hr_and_AB":
            arc_name =arc[:8] + maneuv_indicator
            arc_name_DOY = arc[:8]
        else:
            arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')

        start_arc = pd.to_datetime(OBJECT[choose_model]['run_parameters'+arc_name]['prms_arc']['scaleparameter_times'][0])
        end_arc   = pd.to_datetime(OBJECT[choose_model]['run_parameters'+arc_name]['prms_arc']['scaleparameter_times'][-2])

        # print(end_arc)
        ### SELECT ONLY THE MIDDLE 24-HOURS
        A = OBJECT[choose_model]['Density'][arc_name].query( \
                f"{start_arc.year}"         \
               +f"{start_arc.month:02d}"    \
               +f"{start_arc.day:02d}"      \
               +f"{start_arc.hour:02d}"     \
               +f"{start_arc.minute:02d}"   \
               +f"{start_arc.second:02d}"   \
               +f" <= Date <= "                     \
               +f"{end_arc.year}"       \
               +f"{end_arc.month:02d}"  \
               +f"{end_arc.day:02d}"    \
               +f"{end_arc.hour:02d}"   \
               +f"{end_arc.minute:02d}" \
               +f"{end_arc.second:02d}" \
            )
        
        B = OBJECT[choose_model]['DragFile'][arc_name].query( \
                f"{start_arc.year}"         \
               +f"{start_arc.month:02d}"    \
               +f"{start_arc.day:02d}"      \
               +f"{start_arc.hour:02d}"     \
               +f"{start_arc.minute:02d}"   \
               +f"{start_arc.second:02d}"   \
               +f" <= Date <= "                     \
               +f"{end_arc.year}"       \
               +f"{end_arc.month:02d}"  \
               +f"{end_arc.day:02d}"    \
               +f"{end_arc.hour:02d}"   \
               +f"{end_arc.minute:02d}" \
               +f"{end_arc.second:02d}" \
            )

        len_dates = np.shape(A['rho (kg/m**3)'])[0]

        #24hour period
        for it in np.arange(0,len_dates):
#             den_dict['denscaled'].append(\
#                     OBJECT[choose_model]['Density'][arc]['rho (kg/m**3)'].iloc[it]\
#                     *model_dict[choose_model]['ScalingFactors'][ii])
            den_dict['dens'].append(A['rho (kg/m**3)'].iloc[it])
            den_dict['lon'].append( A['Lon'].iloc[it])
            den_dict['lat'].append( A['Lat'].iloc[it])
            den_dict['alt'].append( A['Height (meters)'].iloc[it])
            den_dict['dates'].append( A['Date'].iloc[it])
            #
            den_dict['CD'].append( B['CD'].iloc[it])

            
    sfdate_list = model_dict[choose_model]['ScalingFactor_times']
    sf_list     = model_dict[choose_model]['ScalingFactors']
    df          = pd.DataFrame.from_dict(den_dict)


    scaled_densities = []
    densities = []
    scaled_dates = []
    scaled_lats = []
    scaled_lons = []
    scaled_alts = []
    scaled_cds = []
    sf = []
    for isf, valsf in enumerate(sfdate_list):
        # print(isf, valsf)
        if isf == 0:
            A=df.query(f" dates <= "              \
                +f"{sfdate_list[isf].year}"       \
                +f"{sfdate_list[isf].month:02d}"  \
                +f"{sfdate_list[isf].day:02d}"    \
                +f"{sfdate_list[isf].hour:02d}"   \
                +f"{sfdate_list[isf].minute:02d}" \
                +f"{sfdate_list[isf].second:02d}" \
                )

        elif isf == len(sfdate_list)-1:
            A=df.query( f"{sfdate_list[isf-1].year}"       \
                    +f"{sfdate_list[isf-1].month:02d}"  \
                    +f"{sfdate_list[isf-1].day:02d}"    \
                    +f"{sfdate_list[isf-1].hour:02d}"   \
                    +f"{sfdate_list[isf-1].minute:02d}" \
                    +f"{sfdate_list[isf-1].second:02d}" \
                    +f" <= dates "                     \
                )
        else:
            # select the values of density corresponding to SF datetimes
            A=df.query( f"{sfdate_list[isf].year}"         \
                    +f"{sfdate_list[isf].month:02d}"    \
                    +f"{sfdate_list[isf].day:02d}"      \
                    +f"{sfdate_list[isf].hour:02d}"     \
                    +f"{sfdate_list[isf].minute:02d}"   \
                    +f"{sfdate_list[isf].second:02d}"   \
                    +f" <= dates <= "                     \
                    +f"{sfdate_list[isf+1].year}"       \
                    +f"{sfdate_list[isf+1].month:02d}"  \
                    +f"{sfdate_list[isf+1].day:02d}"    \
                    +f"{sfdate_list[isf+1].hour:02d}"   \
                    +f"{sfdate_list[isf+1].minute:02d}" \
                    +f"{sfdate_list[isf+1].second:02d}" \
                    )
        ## Apply the scaling factor
        scaled_densities.extend(A['dens'].values*sf_list[isf])
        sf.extend(np.ones(len(A['dens'].values))*sf_list[isf])
        densities.extend(A['dens'].values)
        scaled_dates.extend(A['dates'].values)
        scaled_lats.extend(A['lat'].values)
        scaled_lons.extend(A['lon'].values)
        scaled_alts.extend(A['alt'].values)
        scaled_cds.extend(A['CD'].values)

    den_dict['denscaled'] = scaled_densities
    den_dict['dens']      = densities
    den_dict['sf']        = sf
    den_dict['datescaled'] = pd.to_datetime(scaled_dates)
    den_dict['lat_indx'] = scaled_lats
    den_dict['lon_indx'] = scaled_lons
    den_dict['alt_indx'] = scaled_alts
    den_dict['CD'] = scaled_cds
                
    return(den_dict)

In [6]:
# from pygeodyn.pygeodyn_plot_scalingfactors import *

satid = 1807001
wgts = {}

for model in run_dict.keys():
    wgts[model] = {}
    ScalingFactors  = []
    ScalingFactor_times = []

    for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
        epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
        hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
        frachours =(hrs/24)
        #
        if len(arc) == 9:
            maneuv_indicator = arc[8]
        else:
            maneuv_indicator = ''
        arc_type = obj[model]['global_params']['prms']['arc_type']
        if arc_type == "Nominal30hr_and_AB":
            arc_name =arc[:8]+ maneuv_indicator
            arc_name_DOY = arc[:8]
        else:
            arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        ### Collect the weights for the ensemble average
        inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
        wgts[model][arc_name_DOY] = inv_rms#/sum_wgts

        iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
        for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
            
            numSFs = len(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'])
            
            if iit == 0 or iit==numSFs-1:   # remove the first and last Scaling Factor to account for edge effects
                pass
            else:
                CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                ScalingFactor_times.append(itime)
    run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
    run_dict[model]['ScalingFactors']      = ScalingFactors
    run_dict[model]['Weight'] = wgts[model]
    

    
###### SCALE THE DENSITIES:
models_dens = {}
for monthmodel in run_dict.keys():
    print(f"---Making continuous scaled rho for {monthmodel}")
    models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

    
    
# for monthmodel in run_dict.keys():
    
    

#     del obj[monthmodel]
#     gc_collect()

# del obj
# gc_collect()




---Making continuous scaled rho for 2018octmsis2
---Making continuous scaled rho for 2018novmsis2
---Making continuous scaled rho for 2018octdtm2020_o
---Making continuous scaled rho for 2018novdtm2020_o


In [7]:
    
# for monthmodel in run_dict.keys():
#     models_dens[monthmodel]['CD'] = []
#     obj_m1  = obj[monthmodel]

#     for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
#         epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
#         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#         frachours =(hrs/24)
#         #
#         if len(arc) == 9:
#             maneuv_indicator = arc[8]
#         else:
#             maneuv_indicator = ''
#         arc_type = obj_m1['global_params']['prms']['arc_type']
#         if arc_type == "Nominal30hr_and_AB":
#             arc_name =arc[:8]+ maneuv_indicator
#         else:
#             arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator

#         drag_data        = obj_m1['DragFile'][arc_name]
#         drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
#         den_df           = obj_m1['Density' ][arc_name]

#         ### Cut off the extra time for the regular days
#         start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
#         end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(0, 'm')
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         drag_data = drag_data.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )
#         models_dens[monthmodel]['CD'].extend(list(drag_data['CD'].values))

In [8]:
len(models_dens[monthmodel]['CD'])

256215

In [9]:
print(len(models_dens[monthmodel]['denscaled']))
print(len(models_dens[monthmodel]['datescaled']))

256215
256215


In [10]:
models_dens.keys()

dict_keys(['2018octmsis2', '2018novmsis2', '2018octdtm2020_o', '2018novdtm2020_o'])

# Write to CAMEL

In [11]:
month_list

['oct', 'nov']

In [12]:
monthmodel

'2018novdtm2020_o'

In [13]:
run_list

['msis2', 'dtm2020_o']

In [14]:
run_dict

{'2018octmsis2': {'num': 5,
  'model_path': None,
  'ScalingFactor_times': [Timestamp('2018-10-15 03:00:00'),
   Timestamp('2018-10-15 06:00:00'),
   Timestamp('2018-10-15 09:00:00'),
   Timestamp('2018-10-15 12:00:00'),
   Timestamp('2018-10-15 15:00:00'),
   Timestamp('2018-10-15 18:00:00'),
   Timestamp('2018-10-15 21:00:00'),
   Timestamp('2018-10-16 00:00:00'),
   Timestamp('2018-10-16 03:00:00'),
   Timestamp('2018-10-16 06:00:00'),
   Timestamp('2018-10-16 09:00:00'),
   Timestamp('2018-10-16 12:00:00'),
   Timestamp('2018-10-16 15:00:00'),
   Timestamp('2018-10-16 18:00:00'),
   Timestamp('2018-10-16 21:00:00'),
   Timestamp('2018-10-17 03:00:00'),
   Timestamp('2018-10-17 06:00:00'),
   Timestamp('2018-10-17 09:00:00'),
   Timestamp('2018-10-17 12:00:00'),
   Timestamp('2018-10-17 15:00:00'),
   Timestamp('2018-10-17 18:00:00'),
   Timestamp('2018-10-17 21:00:00'),
   Timestamp('2018-10-18 00:00:00'),
   Timestamp('2018-10-18 03:00:00'),
   Timestamp('2018-10-18 06:00:00'),
  

In [18]:
%load_ext autoreload
%autoreload 2
# from pygeodyn.PYGEODYN import Pygeodyn
from gc import collect as gc_collect

import numpy as np
import pickle 
import os
import pandas as pd
import time
from datetime import datetime,timedelta

# /data/SatDragModelValidation/data/outputs_clean/icesat2/O2R2023_longimeperiod/CAMEL


WRITEFILE = True


global_path    =  '/data/SatDragModelValidation/data/outputs_clean/icesat2/FinalDatasets_SWJ2025/CAMEL/'


### Declare some setup parameters:
len_date        = 19
len_lon_lat_alt =  8
len_cd          =  8
len_rho         = 13
len_resids      = 12
len_f107        = 5
len_kp          = 5


donelist = []

for imodel in run_list:

    if imodel == 'msis2':
        modelname = "NRLMSIS 2.0"
    elif imodel == 'dtm2020_o':
        modelname = "DTM2020 (oper.)"
    elif imodel == 'jb2008':
        modelname = "JB2008"
    elif imodel == 'hasdm_oc':
        modelname = "SET HASDM"

    
    CAMELfilename = global_path+'Oct2018_Apr2019_24hrOrbitFits_wDRIA'+'__'+imodel+'_NOATT.csv'    
    print('   filename: ',CAMELfilename)

    if WRITEFILE:
        ### Delete the existing files so they append cleanly
        if os.path.exists(CAMELfilename):
            os.remove(CAMELfilename)
        f = open(CAMELfilename, "w")
        f.write("\n")
        f.close()

        ### Write to file
        with open(CAMELfilename, 'r+') as file:

            ### Manually write the header units
            header_units =\
                    f"{'(UTC)'.rjust(len_date-1,' ') }"\
                    +f"  {'(deg)'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'(deg)'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'(m)'.rjust(len_lon_lat_alt+5,' ')}"\
                    +f"  {'( )'.rjust(len_cd,' ')}"\
                    +f"  {'( )'.rjust(len_cd,' ')}"\
                    +f"  {'(kg/m^3)'.rjust(len_rho,' ')}"\
                    +f"  {'(kg/m^3)'.rjust(len_rho,' ')}"\
    #                     +f"  {'(m)'.rjust(len_resids,' ')}"\

            #### Manually write the header field names
            header_names =\
                    f"{'Date'.rjust(len_date-1,' ') }"\
                    +f"  {'Lon'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'Lat'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'Alt'.rjust(len_lon_lat_alt+5,' ')}"\
                    +f"  {'CD'.rjust(len_cd,' ')}"\
                    +f"  {'SF'.rjust(len_cd,' ')}"\
                    +f"  {'Rho'.rjust(len_rho,' ')}"\
                    +f"  {'Rho*SF'.rjust(len_rho,' ')}"\
    #                     +f"  {'T_PSO'.rjust(len_resids,' ')}"\

            #### Manually write the detailed header description
            header_meta = \
                f'''### Model Assessment---ICESat-2 Orbit Fits using {modelname}
### -----------------------
###     Satellite: ICESat2
###     Epoch:     +start____ {'2018-10-13 21:18:00'} 
###                +stop_____ {'2019-04-21 02:42:00'}
###     Last modified: {datetime.now()-timedelta(hours=7)}
###
### Data Sources and Notes
### -------
###     ICESat-2 Orbit Fits using {modelname}
###         * 24-hour orbit Fits 
###         * GEODYN STEP size = 10 seconds
###         * Orbit file cadence = 120 seconds
###         * DRIA Params:
###              MS     = 26.980D0 #  molar mass for each panel (g/mol)
###              TW     = 300.0D0  #  temperature of panels  (K)
###              ALPHA  = 0.890D0  #  accomodation coefficient
###              KL     = 0.0D0    #  langmuir parameter
###              FRACOX = 1.0D0    #  fraction of surface covered by atomic oxygen
###
###     ICESat-2 Precise Science Orbits (PSO)
###
###
#{header_units}
#{header_names}
### %eoh
'''

            file.write(header_meta)  
            
            
            #### ONE OF THESE IF STATEMENT SUCKS
            
            for monthmodel in run_dict:
                donelist.append(monthmodel)
                if monthmodel in donelist:
                    pass
                
                elif imodel not in monthmodel :
                    pass
                else:
                    start = time.time()
                    print('\n','Saving Text file for ', monthmodel)

                    dict_dens = models_dens[monthmodel]

                    for itime,timeval in enumerate(dict_dens['dates']):

                        #### Manually write each row of the data.
                        row =   f"{pd.to_datetime(timeval).strftime(format='%Y-%m-%dT%H:%M:%SZ')}"\
                            +f"  {dict_dens['lon_indx'][itime]:9.3f}"\
                            +f"  {dict_dens['lat_indx'][itime]:9.3f}"\
                            +f"  {dict_dens['alt_indx'][itime]:9.3f}"\
                            +f"  {dict_dens['CD'][itime]:9.3f}"\
                            +f"  {dict_dens['sf'][itime]:9.3f}"\
                            +f"  {dict_dens['dens'][itime]:15.6e}"\
                            +f"  {dict_dens['denscaled'][itime]:15.6e}"\
                            +f"\n"
                        file.write(row)
                            #                             +f"  {intrack_PCE:12.5f}"\

                #
                    end = time.time()
                    elapsed = end - start
                    #
                    print()
                    # print(f'       indxes: 0 -',indx,'') 
                    print(f'       Processed file in : ',np.round(elapsed,5),'secs', f"({np.round(elapsed,5)/60} minutes)") 


    else:
        print('fixing file formatting')
        df = pd.read_csv(CAMELfilename, 
                         sep = '\s+',
                         names = [#'DateYMD',
                                  'DateTZ',
                                  'Lon' ,
                                  'Lat' ,
                                  'Alt' ,
                                  'CD' ,
                                  'SF' ,
                                  'Rho' ,
                                  'Rho_scaled' ,
                                ],
                         skiprows = 26)


    
        df.insert(0, 'Date', pd.to_datetime(df['DateTZ'],\
                                            format='%Y-%m-%dT%H:%M:%SZ'))
        del df['DateTZ']
        df = df.dropna().reset_index(drop=True)

        STARTairplane = pd.to_datetime('10-27-2018 13:30:00')
        STOPairplane  = pd.to_datetime('02-26-2019 22:02:00')

        index_airplane = df.query( \
                f"{STARTairplane.year}"         \
               +f"{STARTairplane.month:02d}"    \
               +f"{STARTairplane.day:02d}"      \
               +f"{STARTairplane.hour:02d}"     \
               +f"{STARTairplane.minute:02d}"   \
               +f"{STARTairplane.second:02d}"   \
               +f" < Date <= "                     \
               +f"{STOPairplane.year}"       \
               +f"{STOPairplane.month:02d}"  \
               +f"{STOPairplane.day:02d}"    \
               +f"{STOPairplane.hour:02d}"   \
               +f"{STOPairplane.minute:02d}" \
               +f"{STOPairplane.second:02d}" \
            ).index


        index_sailboat1 = df.query( \
                f"{STARTairplane.year}"         \
               +f"{STARTairplane.month:02d}"    \
               +f"{STARTairplane.day:02d}"      \
               +f"{STARTairplane.hour:02d}"     \
               +f"{STARTairplane.minute:02d}"   \
               +f"{STARTairplane.second:02d}"   \
               +f" >= Date ").index

        index_sailboat2 = df.query(f"Date > "\
               +f"{STOPairplane.year}"       \
               +f"{STOPairplane.month:02d}"  \
               +f"{STOPairplane.day:02d}"    \
               +f"{STOPairplane.hour:02d}"   \
               +f"{STOPairplane.minute:02d}" \
               +f"{STOPairplane.second:02d}" \
            ).index

        df['Mode'] = np.nan

        ### Write flag for airplane mode
        df.loc[index_airplane,'Mode'] = 'A'
        df.loc[index_sailboat1,'Mode'] = 'S'
        df.loc[index_sailboat2,'Mode'] = 'S'
        
        

        CAMELfilename = global_path+'Oct2018_Apr2019_24hrOrbitFits_wDRIA'+'__'+imodel+'.csv'    
        print('   filename: ',CAMELfilename)

        ### Delete the existing files so they append cleanly
        if os.path.exists(CAMELfilename):
            os.remove(CAMELfilename)

        f = open(CAMELfilename, "w")
        f.write("\n")
        f.close()

        ### Write to file
        with open(CAMELfilename, 'r+') as file:

            
            ### Manually write the header units
            header_units =\
                    f"{'(UTC)'.rjust(len_date-1,' ') }"\
                    +f"  {'(deg)'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'(deg)'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'(m)'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'( )'.rjust(len_cd,' ')}"\
                    +f"  {'( )'.rjust(len_cd-2,' ')}"\
                    +f"  {'(kg/m^3)'.rjust(len_rho,' ')}"\
                    +f"  {'(kg/m^3)'.rjust(len_rho,' ')}"\
                    +f"  {'Flag'.rjust(6,' ')}"\
                    +f"  {'Instr'.rjust(6,' ')}"\

            #### Manually write the header field names
            header_names =\
                    f"{'Date'.rjust(len_date-1,' ') }"\
                    +f"  {'Lon'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'Lat'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'Alt'.rjust(len_lon_lat_alt,' ')}"\
                    +f"  {'CD'.rjust(len_cd,' ')}"\
                    +f"  {'SF'.rjust(len_cd-2,' ')}"\
                    +f"  {'Rho'.rjust(len_rho,' ')}"\
                    +f"  {'Rho*SF'.rjust(len_rho,' ')}"\

            #### Manually write the detailed header description
            header_meta = \
            f'''### Model Assessment---ICESat-2 Orbit Fits using {modelname}
### -----------------------
###     Satellite: ICESat2
###     Epoch:     +start____ {'2018-10-13 21:18:00'} 
###                +stop_____ {'2019-04-21 02:42:00'}
###     Last modified: {datetime.now()-timedelta(hours=7)}
###
### Data Sources and Notes
### ----------------------
###     ICESat-2 Orbit Fits using {modelname}
###         * 24-hour orbit fits 
###         * GEODYN STEP size = 10 seconds
###         * Orbit file cadence = 120 seconds
###         * DRIA Params:
###              MS     = 26.980D0 #  molar mass for each panel (g/mol)
###              TW     = 300.0D0  #  temperature of panels  (K)
###              ALPHA  = 0.890D0  #  accommodation coefficient
###              KL     = 0.0D0    #  langmuir parameter
###              FRACOX = 1.0D0    #  fraction of surface covered by atomic oxygen
###
### Variable Details
### -----------------
###
###   VARIABLE        | DETAILS
###   ==========================
###   Date (UTC)      | Date in universal coordinated time   
###   Lon (deg)       | Geodetic Longitude,  0-360
###   Lat (deg)       | Geodetic Latitude,  -90-90
###   Alt (m)         | Geodetic Altitude
###   CD  ( )         | Drag Coefficient as calculated by DRIA using the above noted parameters  
###   SF  ( )         | Scaling Factor determined by GEODYN batch least squares filter  
###   Rho    (kg/m^3) | Model Thermosphere Mass density      
###   Rho*SF (kg/m^3) | Scaled Model Thermosphere Mass density      
###   Flag ( )        | Attitude mode flag ( Airplane (A) or Sailboat (S))
###   Instrument      | Designation for CAMEL
###
###
#{header_units}
#{header_names}
### %eoh
'''

            file.write(header_meta)  

            for i,val in enumerate(df['Date']):
                #### Manually write each row of the data.
                row =   f"{pd.to_datetime(df['Date'][i]).strftime(format='%Y-%m-%dT%H:%M:%SZ')}"\
                +f"  {df['Lon'][i]:8.3f}"\
                +f"  {df['Lat'][i]:8.3f}"\
                +f"  {df['Alt'][i]:9.3f}"\
                +f"  {df['CD'][i]:6.3f}"\
                +f"  {df['SF'][i]:6.3f}"\
                +f"  {df['Rho'][i]:13.6e}"\
                +f"  {df['Rho_scaled'][i]:13.6e}"\
                +f"  {df['Mode'][i]:}"\
                +f"  {'icesat2'}"\
                +f"\n"
                file.write(row)                
#                 +f"  {df['T_PSO'][i]:12.5f}"\

#     WRITEFILE    = False


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
   filename:  /data/SatDragModelValidation/data/outputs_clean/icesat2/FinalDatasets_SWJ2025/CAMEL/Oct2018_Apr2019_24hrOrbitFits_wDRIA__msis2_NOATT.csv
   filename:  /data/SatDragModelValidation/data/outputs_clean/icesat2/FinalDatasets_SWJ2025/CAMEL/Oct2018_Apr2019_24hrOrbitFits_wDRIA__dtm2020_o_NOATT.csv


<>:159: SyntaxWarning: invalid escape sequence '\s'
<>:159: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_26597/1566748374.py:159: SyntaxWarning: invalid escape sequence '\s'
  sep = '\s+',


In [16]:
imodel

'dtm2020_o'

In [17]:
monthmodel


'2018novdtm2020_o'